# Demonstration of validation errors

This notebook showcases how nodes and workflows are validated in the Uncertainty Engine. 
Please note, some cells below intentionally contain errors to illustrate the validation process. You do not need to run all the cells sequentially.

In [1]:
# import client and authenticate
from uncertainty_engine import Client
client = Client()
client.authenticate()

## Client required to perform validation

### Node instantiation without client
Validation requires the `client` to be passed to various components, such as nodes and workflows. Instantiating a node without the required `client` parameter triggers validation warnings. Similar issues can occur if the `client` is omitted elsewhere.

In [2]:
from uncertainty_engine.nodes.base import Node

number_node = Node(
    node_name="Number",
    version="0.2.0",
    value=42,
    label="number_node_without_client",
)

### Node instantiation with client
Instantiate a node with the `client` parameter to get node parameter validation whilst building a workflow.

In [3]:
number_node = Node(
    node_name="Number",
    version="0.2.0",
    value=42,
    label="number_node_with_client",
    client=client,
)

## Node instantiation

Below is a comparison between using a custom node class versus the base `Node` class. Using the base `Node` class to instantiate a node with the wrong parameters will cause validation errors. Including the `client` in the specific node classes, will be useful for the handle validation.

In [4]:
# using an existing custom class
from uncertainty_engine.nodes.basic import Add

first_add = Add(
    lhs=5,
    rhs=10,
    label="first_add_node",
    client=client,
)

In [5]:
# using the base Node class
from uncertainty_engine.nodes.base import Node

second_add = Node(
    node_name="Add",
    version="0.2.0",
    lhs=5,
    label="second_add_node",
    client=client,
)

NodeValidationError: Missing required inputs: ['rhs']

Using the base `Node` class to instantiate a node type that does not exist will raise a "404: not found" error.

In [6]:
my_node = Node(
    node_name="My Node",
    version="0.2.0",
    my_param=1,
    label="my_node",
    client=client,
)

HTTPError: 404 Not Found: The node 'My Node' does not exist.

## Handle creation

Handles should reference existing node outputs. Using `make_handle` on a node instantiated with the client will raise an error if a handle name is incorrect and it will list any available outputs for that node.

In [7]:
second_add = Node(
    node_name="Add",
    version="0.2.0",
    lhs=5,
    rhs=2,
    label="second_add_node",
    client=client,
)

In [8]:
name_handle = second_add.make_handle("sum")

NodeValidationError: Invalid output names: ['sum']. Please make a handle using any of the following outputs instead: ['ans'].

In [9]:
ans_handle = second_add.make_handle("ans")

## Adding a node to the graph

Adding nodes to a graph triggers validation if the `prevent_node_overwrite` argument is set to `True`. If a node with a duplicate `label` is added, an exception is raised.

In [10]:
# import graph and instantiate
from uncertainty_engine.graph import Graph
graph = Graph(prevent_node_overwrite=True)

In [11]:
number_node_five = Node(
    node_name="Number",
    version="0.2.0",
    value=5,
    label="number_node",
    client=client,
)

graph.add_node(number_node_five)

In [12]:
number_node_one = Node(
    node_name="Number",
    version="0.2.0",
    value=1,
    label="number_node",
    client=client,
)

graph.add_node(number_node_one)

GraphValidationError: Label 'number_node' already used in the graph

## Validating the workflow

Constructing a workflow with problematic or incomplete inputs and passing the `client` triggers validation errors. The workflow performs a full validation of all included nodes and their handles, even if some nodes were instantiated without a `client`. Errors will be raised for any invalid inputs that could cause the workflow to fail at runtime.

In [13]:
# create a new graph
graph_with_validation_problems = Graph()

In [14]:
# add some nodes
one_node = Node(
    node_name="Number",
    version="0.2.0",
    value=1,
    label="one_node",
    client=client,
)

add_node = Add(
    lhs=one_node.make_handle("value"),
    rhs=2,
    label="add_node",
    client=client,
)

graph_with_validation_problems.add_node(one_node)
graph_with_validation_problems.add_node(add_node)

In [15]:
# create a workflow from the graph
from uncertainty_engine.nodes.workflow import Workflow

workflow = Workflow(
    graph=graph_with_validation_problems.nodes,
    inputs=graph_with_validation_problems.external_input,
    external_input_id=graph_with_validation_problems.external_input_id,
    requested_output={
        "Addition result": {
            "node_name": "Add",
            "node_handle": "ans",
        },
        "Addition result alternative": add_node.make_handle("ans"),
    },
    client=client,
)

WorkflowValidationError: Workflow Validation Failed

Requested Output Errors:
  - Addition result: Node with label 'Add' is referenced but is not in graph.
  - Addition result alternative: Requested output must be a dictionary, not a `Handle` object. Did you mean to use `handle.model_dump()`?